## Step by step buidling of a knowledge graph

The goal of this notebook is to build an example of knowledge graph using a step by step approach.


## Prerequisites

This notebook assumes you've created a project within the sandbox deployment of Nexus. If not Go to https://sandbox.bluebrainnexus.io/web/ to create a project and obtain a valid authentication login.

## Overview
Explain the Research domain (with figure). Uses schema.org and json-ld to describe entities

You'll work through the following steps:

0. Clone the tuorial Github project (Google Colab only)
1. Create and configure a Blue Brain Nexus client
2. Create a Person entity
3. Create an Organization entity and link it to the Person entity as affiliation
4. Create an article entity and link it to the Person entity as author
5. Explore and navigate the created knowledge graph

## Step 0: Clone the tuorial Github project (Google Colab only)

### Remove the comments and run the code

In [ ]:
"""
import os 

tutorial_base_dir = "/content/nexus-bbp-domains"
if os.path.exists(tutorial_base_dir):
  !rm -R $tutorial_base_dir

!git clone --single-branch --branch docs https://github.com/BlueBrain/nexus-bbp-domains.git
os.chdir("/".join([tutorial_base_dir,"src/main/paradox/docs/bluebrainnexustutorialkcni/notebooks"]))

print("The working directory is now:")
!pwd
"""

## Step 1: Create and configure a Nexus client

In [1]:
#install the Blue Brain Nexus python SDK
!pip install -U nexus-sdk

Requirement already up-to-date: nexus-sdk in /Users/mfsy/anaconda3/envs/kgforge_is_awesome/lib/python3.7/site-packages (0.3.2)


In [2]:
#install SPARQLWrapper
!pip install -U validators

Requirement already up-to-date: SPARQLWrapper in /Users/mfsy/anaconda3/envs/kgforge_is_awesome/lib/python3.7/site-packages (1.8.5)


In [ ]:
#install SPARQLWrapper
!pip install -U SPARQLWrapper

In [4]:
#Set a token to authenticate to Nexus
import getpass
token = getpass.getpass()

········


### Create a nexus client. Don't forget to replace \$PROJECTLABEL with your own project label (project=...)

In [5]:
#Configure a nexus client
nexus_environment = "https://sandbox.bluebrainnexus.io/v1"
org ="tutorialnexus"
project ="test_switch"

import nexussdk as nexus
nexus.config.set_environment(nexus_environment)
nexus.config.set_token(token)

#The project default vocabulary
vocab = "%s/vocabs/%s/%s/"%(nexus_environment, org, project)

In [11]:
from pygments.lexers import JsonLdLexer
from pygments import highlight
from pygments.formatters import TerminalFormatter, TerminalTrueColorFormatter
import json
import requests
import rdflib, json
from rdflib.extras.external_graph_libs import rdflib_to_networkx_digraph
import validators
import networkx as nx
from pyvis.network import Network as pyvisnx
from pyvis import network as net

In [12]:
from sparqlendpointhelper import SparqlViewHelper

%load_ext autoreload
%autoreload 1
%aimport sparqlendpointhelper
%aimport utils
%reload_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Create a SPARQL helper to be used for running queries 

In [8]:
sparqlview_endpoint = nexus_environment+"/views/"+org+"/"+project+"/graph/sparql"
sparqlviewhelper = SparqlViewHelper(sparqlview_endpoint,nexus_environment, org, project, token)

### Import the schema.org context

The following calls will create a jsonld context (identified by http://schema.org) resource importing the schema.org context available [here](https://schema.org/docs/jsonldcontext.json).

In [23]:
schemaorg_context_address = "https://schema.org/docs/jsonldcontext.json"
response = requests.get(schemaorg_context_address)
schemaorg_context = response.json()
#print(schemaorg_context)
schemaorg_context["@id"]="http://schema.org"
create_response = utils.create_resource(nexus,schemaorg_context,org,project)
utils.pretty_print(create_response)


{
    "@context": "https://bluebrain.github.io/nexus/contexts/resource.json",
    "@id": "http://schema.org",
    "_self": "https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/test_switch/_/http%3A%2F%2Fschema.org",
    "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/unconstrained.json",
    "_project": "https://sandbox.bluebrainnexus.io/v1/projects/tutorialnexus/test_switch",
    "_rev": 1,
    "_deprecated": false,
    "_createdAt": "2020-05-04T23:39:23.631532Z",
    "_createdBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy",
    "_updatedAt": "2020-05-04T23:39:23.631532Z",
    "_updatedBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy",
    "_incoming": "https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/test_switch/_/http%3A%2F%2Fschema.org/incoming",
    "_outgoing": "https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/test_switch/_/http%3A%2F%2Fschema.org/outgoing"
}



## Step 2: Create a Person entity

Let define an entity of type Person as follows:

* A person is an entity of type Person (@type value)
* A person has an identifier (@id value)
* A person has a family name (familyName value), a given name (givenName value) and a job title (jobTitle value)
* A person has a job


In [29]:
# Use an orcid identifier if you have one, or your github id
person ={
    "@context":"http://schema.org",
    "@id":"http://your/id/heresssss",
    "@type":"Person",
    "familyName":"your familly name here",
    "givenName":"your given name here",
    "jobTitle":"job title"
} 

response = utils.create_resource(nexus,person,org,project)
utils.pretty_print(response)


{
    "@context": "https://bluebrain.github.io/nexus/contexts/resource.json",
    "@id": "http://your/id/heresssss",
    "@type": "http://schema.org/Person",
    "_self": "https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/test_switch/_/http%3A%2F%2Fyour%2Fid%2Fheresssss",
    "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/unconstrained.json",
    "_project": "https://sandbox.bluebrainnexus.io/v1/projects/tutorialnexus/test_switch",
    "_rev": 1,
    "_deprecated": false,
    "_createdAt": "2020-05-05T10:47:30.881579Z",
    "_createdBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy",
    "_updatedAt": "2020-05-05T10:47:30.881579Z",
    "_updatedBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy",
    "_incoming": "https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/test_switch/_/http%3A%2F%2Fyour%2Fid%2Fheresssss/incoming",
    "_outgoing": "https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/test_switch/_/ht

## Step 3: Create an article entity and link it to the Person entity as author

The knowledge Graph now contains a single entity of type Person. Let add to the knowledge graph one scholarly article (publication) authored by the person entity:

* A scholarly article is  an entity of type ScholarlyArticle (@type value)
* A scholarly article has an identifier (@id value)
* A scholarly article has a name (name value)
* A scholarly article has a publishing date (datePublished value)


###  Create a ScholarlyArticle entity

In [30]:
#Let create an entity describing a publication with a doi: https://doi.org/10.1186/1471-2105-13-s1-s4
scholarly_article ={
    "@context":"http://schema.org",
    "@type":"ScholarlyArticle",
    "@id":"https://doi.org/10.1016/j.cell.2015.09.029",
    "name":"Reconstruction and Simulation of Neocortical Microcircuitry",
    "datePublished":"2015-10"
} 

response = utils.create_resource(nexus,scholarly_article,org,project)
utils.pretty_print(response)

{"@context":"https://bluebrain.github.io/nexus/contexts/error.json","@type":"ResourceAlreadyExists","reason":"Resource 'https://doi.org/10.1016/j.cell.2015.09.029' already exists."}
{
    "@context": "https://bluebrain.github.io/nexus/contexts/error.json",
    "@type": "ResourceAlreadyExists",
    "reason": "Resource 'https://doi.org/10.1016/j.cell.2015.09.029' already exists."
}



###  Link the Person and the ScholarlyArticle entity with authorship

In [31]:
# A reference to the Person identifier (value of @id) is enough to link with the article
# Note the revision value change (should be "_rev": 2) after an update
scholarly_article["author"]={
    "@id":person["@id"]
}
response = utils.update_resource(nexus,scholarly_article["@id"],scholarly_article,org,project)
utils.pretty_print(response)

{
    "@context": "https://bluebrain.github.io/nexus/contexts/resource.json",
    "@id": "https://doi.org/10.1016/j.cell.2015.09.029",
    "@type": "http://schema.org/ScholarlyArticle",
    "_self": "https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/test_switch/_/https%3A%2F%2Fdoi.org%2F10.1016%2Fj.cell.2015.09.029",
    "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/unconstrained.json",
    "_project": "https://sandbox.bluebrainnexus.io/v1/projects/tutorialnexus/test_switch",
    "_rev": 2,
    "_deprecated": false,
    "_createdAt": "2020-05-05T10:47:18.771804Z",
    "_createdBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy",
    "_updatedAt": "2020-05-05T10:47:39.610655Z",
    "_updatedBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy",
    "_incoming": "https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/test_switch/_/https%3A%2F%2Fdoi.org%2F10.1016%2Fj.cell.2015.09.029/incoming",
    "_outgoing": "https://sandb

###  Fetch the ScholarlyArticle by identifier to view the update 

In [32]:
response = utils.fetch_resource(nexus,scholarly_article["@id"],org,project)
utils.pretty_print(response)

{
    "@context": [
        "http://schema.org",
        "https://bluebrain.github.io/nexus/contexts/resource.json"
    ],
    "author": {
        "id": "http://your/id/heresssss"
    },
    "datePublished": "2015-10",
    "id": "https://doi.org/10.1016/j.cell.2015.09.029",
    "name": "Reconstruction and Simulation of Neocortical Microcircuitry",
    "type": "ScholarlyArticle",
    "_self": "https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/test_switch/_/https%3A%2F%2Fdoi.org%2F10.1016%2Fj.cell.2015.09.029",
    "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/unconstrained.json",
    "_project": "https://sandbox.bluebrainnexus.io/v1/projects/tutorialnexus/test_switch",
    "_rev": 2,
    "_deprecated": false,
    "_createdAt": "2020-05-05T10:47:18.771804Z",
    "_createdBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy",
    "_updatedAt": "2020-05-05T10:47:39.610655Z",
    "_updatedBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users

In [48]:
#Let create an entity describing a publication with a doi: https://doi.org/10.1186/1471-2105-13-s1-s4
organisation ={
    "@context":"http://schema.org",
    "@type":"Organization",
    "@id":"https://www.grid.ac/institutes/grid.5333.6",
    "identifier":"grid.5333.6",
    "name":"École Polytechnique Fédérale de Lausanne",
    "acronym":"EPFL"
} 

response = utils.create_resource(nexus,organisation,org,project)
utils.pretty_print(response)

{
    "@context": "https://bluebrain.github.io/nexus/contexts/resource.json",
    "@id": "https://www.grid.ac/institutes/grid.5333.6",
    "@type": "http://schema.org/Organization",
    "_self": "https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/test_switch/_/https%3A%2F%2Fwww.grid.ac%2Finstitutes%2Fgrid.5333.6",
    "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/unconstrained.json",
    "_project": "https://sandbox.bluebrainnexus.io/v1/projects/tutorialnexus/test_switch",
    "_rev": 2,
    "_deprecated": false,
    "_createdAt": "2020-05-05T10:57:44.422943Z",
    "_createdBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy",
    "_updatedAt": "2020-05-05T11:03:52.955168Z",
    "_updatedBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy",
    "_incoming": "https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/test_switch/_/https%3A%2F%2Fwww.grid.ac%2Finstitutes%2Fgrid.5333.6/incoming",
    "_outgoing": "https://sandbox.b

In [41]:
#Let display all triples

all_triples = """
Select *
 WHERE  {
    ?s ?p ?o.
    
    FILTER (isURI(?p) && ! STRSTARTS(str(?p), "https://bluebrain.github.io/nexus/vocabulary/" ) ).
    FILTER (isURI(?s) && ! STRSTARTS(str(?s), "https://bluebrain.github.io/nexus/vocabulary/" ) ).
    
} LIMIT 20
"""

results_df = sparqlviewhelper.query_sparql(org_with_properties,result_format = "DATAFRAME")
display(results_df.head(100))


,s,p,o
0,https://doi.org/10.1016/j.cell.2015.09.029,http://schema.org/datePublished,2015-10
1,https://doi.org/10.1016/j.cell.2015.09.029,http://schema.org/name,Reconstruction and Simulation of Neocortical Microcircuitry
2,https://doi.org/10.1016/j.cell.2015.09.029,http://schema.org/author,http://your/id/heresssss
3,https://doi.org/10.1016/j.cell.2015.09.029,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://schema.org/ScholarlyArticle
4,http://your/id/heresssss,http://schema.org/familyName,your familly name here
5,http://your/id/heresssss,http://schema.org/givenName,your given name here
6,http://your/id/heresssss,http://schema.org/jobTitle,job title
7,http://your/id/heresssss,http://schema.org/affiliation,https://www.grid.ac/institutes/grid.5333.6
8,http://your/id/heresssss,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://schema.org/Person
9,https://www.grid.ac/institutes/grid.5333.6,http://schema.org/name,École Polytechnique Fédérale de Lausanne


##  Step 4: Update the Person entity to add its affiliation

Let update the person entity with an affiliation information. We will use EPFL as an affiliation and link to the person entity via "affiliation" propoerty.

In this step, we'll:

* Search for the EPFL organization entity and retrieve its identifier
* Update the person entity affiliation to point to the EPFL entity

### Search for the EPFL organization entity and retrieve its identifier and name

The query below is a SPARQL query. Most SPARQL queries you'll see will have the anotomy above with:

* a SELECT clause that let you select the variables you want to retrieve
* a WHERE clause defining a set of constraints that the variables should satisfy to be retrieved
* LIMIT and OFFSET clauses to enable pagination
* the constraints are usually graph patterns in the form of triple (?s for subject, ?p for property and ?o for ?object)

In [49]:
acronym = "\"EPFL\""


epfl_query = """
Select  ?institute  ?name ?grid_id
WHERE {
    ?institute schema:acronym %s.
    ?institute a schema:Organization.
    ?institute schema:identifier ?grid_id . 
    ?institute schema:name ?name . 
}
LIMIT 100
""" % (acronym)

orgs_df = sparqlviewhelper.query_sparql(epfl_query,result_format = "DATAFRAME")
display(orgs_df.head(100))

,institute,name,grid_id
0,https://www.grid.ac/institutes/grid.5333.6,École Polytechnique Fédérale de Lausanne,grid.5333.6


### Update the person entity affiliation to point to the EPFL entity

In [50]:
# Note the revision value change after an update
person["affiliation"]=orgs_df["institute"]

for index, row in orgs_df.iterrows():
    person["affiliation"]={
        "@id":row["institute"]
    }

response = utils.update_resource(nexus,person["@id"],person,org,project)
utils.pretty_print(response)

{
    "@context": "https://bluebrain.github.io/nexus/contexts/resource.json",
    "@id": "http://your/id/heresssss",
    "@type": "http://schema.org/Person",
    "_self": "https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/test_switch/_/http%3A%2F%2Fyour%2Fid%2Fheresssss",
    "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/unconstrained.json",
    "_project": "https://sandbox.bluebrainnexus.io/v1/projects/tutorialnexus/test_switch",
    "_rev": 2,
    "_deprecated": false,
    "_createdAt": "2020-05-05T10:47:30.881579Z",
    "_createdBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy",
    "_updatedAt": "2020-05-05T11:04:59.578864Z",
    "_updatedBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy",
    "_incoming": "https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/test_switch/_/http%3A%2F%2Fyour%2Fid%2Fheresssss/incoming",
    "_outgoing": "https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/test_switch/_/ht

In [42]:
#Let display all triples

all_triples = """
Select *
 WHERE  {
    ?s ?p ?o.
    
    FILTER (isURI(?p) && ! STRSTARTS(str(?p), "https://bluebrain.github.io/nexus/vocabulary/" ) ).
    FILTER (isURI(?s) && ! STRSTARTS(str(?s), "https://bluebrain.github.io/nexus/vocabulary/" ) ).
    
} LIMIT 20
"""

results_df = sparqlviewhelper.query_sparql(org_with_properties,result_format = "DATAFRAME")
display(results_df.head(100))


,s,p,o
0,http://your/id/heresssss,http://schema.org/familyName,your familly name here
1,http://your/id/heresssss,http://schema.org/givenName,your given name here
2,http://your/id/heresssss,http://schema.org/jobTitle,job title
3,http://your/id/heresssss,http://schema.org/affiliation,https://www.grid.ac/institutes/grid.5333.6
4,http://your/id/heresssss,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://schema.org/Person
5,https://www.grid.ac/institutes/grid.5333.6,http://schema.org/name,École Polytechnique Fédérale de Lausanne
6,https://www.grid.ac/institutes/grid.5333.6,http://schema.org/identifier,grid.5333.6
7,https://www.grid.ac/institutes/grid.5333.6,http://schema.org/acronym,EPFL
8,https://www.grid.ac/institutes/grid.5333.6,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://schema.org/Organization
9,https://doi.org/10.1016/j.cell.2015.09.029,http://schema.org/datePublished,2015-10


## Setp 5 Explore and navigate the created knowledge graph using the SPARQL query language

Let write our first query.

In [51]:
select_all_query = """
SELECT ?s ?p ?o
WHERE
{
  ?s ?p ?o.
  
}
OFFSET 0
LIMIT 5
"""

results_df = sparqlviewhelper.query_sparql(select_all_query,result_format = "DATAFRAME")

results_df.head()

,s,p,o
0,https://bluebrain.github.io/nexus/vocabulary/defaultInProject,https://bluebrain.github.io/nexus/vocabulary/constrainedBy,https://bluebrain.github.io/nexus/schemas/resolver.json
1,https://bluebrain.github.io/nexus/vocabulary/defaultInProject,https://bluebrain.github.io/nexus/vocabulary/createdAt,2020-05-04T23:24:51.850Z
2,https://bluebrain.github.io/nexus/vocabulary/defaultInProject,https://bluebrain.github.io/nexus/vocabulary/createdBy,https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy
3,https://bluebrain.github.io/nexus/vocabulary/defaultInProject,https://bluebrain.github.io/nexus/vocabulary/deprecated,false
4,https://bluebrain.github.io/nexus/vocabulary/defaultInProject,https://bluebrain.github.io/nexus/vocabulary/incoming,https://sandbox.bluebrainnexus.io/v1/resolvers/tutorialnexus/test_switch/defaultResolver/incoming


Most SPARQL queries you'll see will have the anotomy above with:
* a **SELECT** clause that let you select the variables you want to retrieve
* a **WHERE** clause defining a set of constraints that the variables should satisfy to be retrieved
* **LIMIT** and **OFFSET** clauses to enable pagination
* the constraints are usually graph patterns in the form of **triple** (?s for subject, ?p for property and ?o for ?object)

Multiple triples can be provided as graph pattern to match but each triple should end with a period. As an example, let retrieve EPFL (?institute) along with its name (?name) and identifier (?grid_id).

In [54]:
epfl_with_name = """

Select  ?institute ?name ?grid_id
WHERE {
    ?institute schema:acronym "EPFL".
    ?institute a schema:Organization.
    ?institute schema:identifier ?grid_id . 
    ?institute schema:name ?name . 
}
LIMIT 100
"""

results_df = sparqlviewhelper.query_sparql(epfl_with_name,result_format = "DATAFRAME")
results_df.head()


,institute,name,grid_id
0,https://www.grid.ac/institutes/grid.5333.6,École Polytechnique Fédérale de Lausanne,grid.5333.6


This is a typical instance query where entities are filtered by their type(s) and then some of their properties are retrieved (here ?name). 

Let retrieve everything that is linked (outgoing) to the organizations. 
The * character in the SELECT clause indicates to retreve all variables: ?institute, ?p, ?o

In [57]:
org_with_properties = """
Select *
 WHERE  {
    ?institute a schema:Organization.
    ?institute ?p ?o.
} LIMIT 20
"""

results_df = sparqlviewhelper.query_sparql(org_with_properties,result_format = "DATAFRAME")
results_df.head(20)


,institute,p,o
0,https://www.grid.ac/institutes/grid.5333.6,https://bluebrain.github.io/nexus/vocabulary/constrainedBy,https://bluebrain.github.io/nexus/schemas/unconstrained.json
1,https://www.grid.ac/institutes/grid.5333.6,https://bluebrain.github.io/nexus/vocabulary/createdAt,2020-05-05T10:57:44.422Z
2,https://www.grid.ac/institutes/grid.5333.6,https://bluebrain.github.io/nexus/vocabulary/createdBy,https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy
3,https://www.grid.ac/institutes/grid.5333.6,https://bluebrain.github.io/nexus/vocabulary/deprecated,false
4,https://www.grid.ac/institutes/grid.5333.6,https://bluebrain.github.io/nexus/vocabulary/incoming,https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/test_switch/_/https%3A%2F%2Fwww.grid.ac%2Finstitutes%2Fgrid.5333.6/incoming
5,https://www.grid.ac/institutes/grid.5333.6,https://bluebrain.github.io/nexus/vocabulary/outgoing,https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/test_switch/_/https%3A%2F%2Fwww.grid.ac%2Finstitutes%2Fgrid.5333.6/outgoing
6,https://www.grid.ac/institutes/grid.5333.6,https://bluebrain.github.io/nexus/vocabulary/project,https://sandbox.bluebrainnexus.io/v1/projects/tutorialnexus/test_switch
7,https://www.grid.ac/institutes/grid.5333.6,https://bluebrain.github.io/nexus/vocabulary/rev,2
8,https://www.grid.ac/institutes/grid.5333.6,https://bluebrain.github.io/nexus/vocabulary/self,https://sandbox.bluebrainnexus.io/v1/resources/tutorialnexus/test_switch/_/https%3A%2F%2Fwww.grid.ac%2Finstitutes%2Fgrid.5333.6
9,https://www.grid.ac/institutes/grid.5333.6,https://bluebrain.github.io/nexus/vocabulary/updatedAt,2020-05-05T11:03:52.955Z


As a little exercise, write a query retrieving incoming entities to organizations. You can copy past the query above and modify it.

Hints: ?s ?p ?o can be read as: ?o is linked to ?s with an outgoing link.

Do you have results ?

In [ ]:
#Your query here
